In [20]:
import numpy as np
from numba import jit
from skimage.util import view_as_blocks

def downsample_labels_nd(a, factor):
    """
    Downsample the given ND array by binning the pixels into bins
    sized according to the downsampling 'factor'.
    
    Each bin is reduced to one pixel in the output array, whose value
    is the most common element from the bin.

    The blockshape can be given by 'factor', which must be either an int or tuple.
    If factor is tuple, it defines the blockshape.
    If factor is an int, then the blockshape will be square (or cubic, etc.)
    """
    @jit(nopython=True)
    def _flat_mode_destructive(data):
        """
        Given a flat array, return the mode.
        Beware: Overwrites flat_data.
        
        Note: We could have used scipy.stats.mode() here,
              but that implementation is insanely slow for large arrays,
              especially if there are many label values in the array.
        """
        data.sort()
        diff = np.diff(data)
        diff_bool = np.ones((len(diff)+2,), dtype=np.uint8)
        diff_bool[1:-1] = (diff != 0)
    
        diff_nonzero = diff_bool.nonzero()[0]
        run_lengths = diff_nonzero[1:] - diff_nonzero[:-1]
        max_run = np.argmax(run_lengths)
        return data[diff_nonzero[max_run]]
    
    @jit(nopython=True)
    def _blockwise_modes(data, output_shape):
        """
        Given an array of shape = output_shape + block_shape,
        compute the mode of each block.
        """
        assert len(output_shape) == len(data.shape) / 2
        
        # This assertion not allowed in nopython mode, but it's correct.
        #assert output_shape == data.shape[:len(data.shape)] 
        
        modes = np.zeros(output_shape, data.dtype)
        for block_index in np.ndindex(output_shape):
            flat_block_data = data[block_index].copy().reshape(-1)
            modes[block_index] = _flat_mode_destructive(flat_block_data)
        return modes

    # If the factor is an int, convert it to a blockshape.
    if np.issubdtype(type(factor), np.integer):
        blockshape = (factor,)*a.ndim
    else:
        blockshape = factor

    assert not (np.array(a.shape) % blockshape).any(), \
        "Downsampling factor must divide cleanly into array shape"

    v = view_as_blocks(a, blockshape)
    return _blockwise_modes(v, v.shape[:a.ndim])


In [21]:
a = np.random.randint(0,4, (16,16))
print a

[[1 1 2 3 0 0 2 3 3 3 3 1 2 1 2 1]
 [3 1 2 2 1 1 1 3 0 1 0 3 2 3 0 0]
 [0 1 1 0 2 1 2 1 2 3 1 0 1 0 2 3]
 [2 2 3 2 1 1 1 3 0 2 0 1 1 1 0 3]
 [1 1 1 0 0 0 1 3 1 3 1 1 1 0 3 0]
 [2 0 1 1 1 1 1 1 0 0 3 1 0 0 2 2]
 [0 0 1 0 2 3 3 3 1 3 3 1 1 0 1 0]
 [2 0 3 0 3 3 3 1 2 1 1 3 3 1 2 1]
 [2 1 0 0 0 1 2 0 0 2 1 0 3 2 3 2]
 [3 3 2 2 3 0 1 0 0 1 3 1 1 1 3 1]
 [1 2 3 0 2 2 1 1 3 2 0 3 1 1 0 2]
 [1 0 3 3 3 3 1 3 1 0 3 3 2 0 2 0]
 [0 1 1 1 3 1 0 3 2 2 3 2 1 2 3 2]
 [2 1 0 1 3 3 0 3 1 1 3 3 2 1 3 3]
 [2 2 2 2 2 1 1 1 0 1 1 2 2 2 0 1]
 [1 2 2 2 2 2 2 1 0 2 1 0 0 0 3 2]]


In [22]:
print downsample_labels_nd(a, 2)

[[1 2 0 3 3 3 2 0]
 [2 0 1 1 2 0 1 3]
 [1 1 0 1 0 1 0 2]
 [0 0 3 3 1 1 1 1]
 [3 0 0 0 0 1 1 3]
 [1 3 2 1 0 3 1 0]
 [1 1 3 0 1 3 1 3]
 [2 2 2 1 0 1 0 0]]


In [23]:
a = np.random.randint(0,4, (6,8)).view(np.uint64)
print a

[[1 1 3 3 1 2 0 3]
 [0 1 1 2 0 0 1 0]
 [1 2 1 2 0 2 3 0]
 [3 0 2 2 0 1 0 3]
 [2 2 3 2 3 2 1 0]
 [3 3 3 0 2 0 2 2]]


In [24]:
print downsample_labels_nd(a, (3,4))

[[1 0]
 [2 0]]


In [25]:
a = np.random.randint(0,4, (4,6,8)).view(np.uint64)
print a

[[[0 0 2 1 0 2 3 2]
  [0 0 0 3 1 3 2 3]
  [3 0 1 1 2 2 2 3]
  [2 0 1 3 0 2 3 3]
  [1 1 0 3 3 2 0 3]
  [1 3 1 2 0 2 0 2]]

 [[0 1 1 2 2 1 1 3]
  [0 1 1 0 0 3 1 0]
  [2 0 2 1 1 1 3 3]
  [0 2 1 3 0 1 1 2]
  [3 0 2 3 1 3 0 0]
  [1 3 3 0 3 0 0 1]]

 [[1 3 3 2 2 3 3 1]
  [3 3 0 2 1 0 1 2]
  [3 0 2 0 0 0 1 3]
  [2 3 2 3 1 0 0 0]
  [0 0 3 1 3 0 3 2]
  [0 2 3 1 1 1 1 0]]

 [[3 2 0 3 2 3 3 0]
  [1 2 0 0 2 1 2 1]
  [0 2 2 2 1 3 2 3]
  [0 1 3 1 2 3 3 2]
  [3 3 0 3 3 0 0 1]
  [3 2 0 3 2 2 2 2]]]


In [26]:
print downsample_labels_nd(a, (2,3,4))

[[[0 3]
  [3 0]]

 [[2 1]
  [3 0]]]
